In [ ]:
# with hugging face wrapper for sanity check
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from ast import literal_eval
import re

In [ ]:
#training hyperparameters
MAX_TOKENS = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05

# change to true to run per review
EXPANDED = False

model_checkpoint = "distilbert/distilbert-base-uncased"

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
review_df = pd.read_csv('../data/split/train.csv', usecols=['Overall Compliance', 'reviews'])
test_df = pd.read_csv('../data/split/test.csv', usecols=['Overall Compliance', 'reviews'])
review_df['reviews'] = review_df['reviews'].apply(literal_eval)
test_df['reviews'] = test_df['reviews'].apply(literal_eval)

# test classifying at reivew level then resturant level
if EXPANDED:
    review_df = review_df.explode('reviews')
    review_df = review_df.reset_index().drop(columns=['index'])

    test_df = test_df.explode('reviews')
    test_df = test_df.reset_index().drop(columns=['index'])

train_ds = Dataset.from_pandas(review_df)
test_df = Dataset.from_pandas(test_df)

review_dataset = DatasetDict()

review_dataset['train'] = train_ds
review_dataset['test'] = test_df

In [ ]:
def clean_reviews(reviews):
    cleaned = []
    for review in reviews:
        review = review.replace('\n', ' ')
        cleaned.append(re.sub(r"[^a-zA-Z0-9]", ' ', review).strip()) #may need to find a better way to do so

    return cleaned

def extract_text_features(data):
    output = {}
    reviews = clean_reviews(data['reviews'])
    output['text'] = " ".join(reviews)
    output['label'] = 0 if data['Overall Compliance'] == 'Yes' else 1

    return output


def extract_expanded_text_features(data):
    output = {}
    review = data['reviews']
    review = review.replace('\n', ' ')
    output['text'] = re.sub(r"[^a-zA-Z0-9]", ' ', review).strip()
    output['label'] = 0 if data['Overall Compliance'] == 'Yes' else 1

    return output

In [ ]:
extract_function = extract_text_features

if EXPANDED:
    extract_function = extract_expanded_text_features

review_dataset = review_dataset.map(extract_function)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize(examples):
    return tokenizer(examples["text"], max_length=MAX_TOKENS, truncation=True)

In [ ]:
token_dataset = review_dataset.map(tokenize, remove_columns=['Overall Compliance', 'reviews'])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "Pass", 1: "Fail"}
label2id = {"Pass": 0, "Fail": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

training_args = TrainingArguments(
    output_dir="base_bert_model",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_dataset["train"],
    eval_dataset=token_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()